# 张量运算
## 逐元素运算
rule 运算和 加法 都是逐元素（element-wise）运算，即该运算独立地应用于张量中的每个元素。
### rule 运算的简单实现

In [32]:
import numpy as np


def sugar_rule(x: np.ndarray):
    assert len(x.shape) == 2  #x 是一个 Numpy的2D张量
    res = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            res[i, j] = max(x[i, j], 0)
    return res


v = np.array([[1, 3], [4, -1]])
print("sugar_rule")
res1 = sugar_rule(v)
print(res1)
print("numpy rule:")
print(np.maximum(v, 0))


sugar_rule
[[1 3]
 [4 0]]
numpy rule:
[[1 3]
 [4 0]]


### 加法的简单实现

In [33]:
def sugar_add(x: np.ndarray, y: np.ndarray):
    assert len(x.shape) == 2
    assert x.shape==y.shape
    res = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            res[i,j]+=y[i,j]
    return res
a = np.array([[1, 3], [4, -1]])
b = np.array([[1, 5], [7, 9]])
print("sugar_add:")
print(sugar_add(a,b))
print("numpy add :")
print(a+b)

sugar_add:
[[ 2  8]
 [11  8]]
numpy add :
[[ 2  8]
 [11  8]]


### 广播
在👆的例子中，add是两个形状相同的张量相加，如何两个形状不同的张量相加：
如果没有歧义的话，较小的张量会被广播，以匹配较大张量的形状。广播包含以下两步。
+ 向较小的张量添加轴，使其ndim与较大的张量相同。
+ 将较小的张量沿着新轴重复，使其形状与较大的张量相同。

一个例子：
假设X的形状是（32，10），y的形状是（10,）。首先，给y添加空的第一个轴，
这样y的形状为变为（1，10）。然后，将y沿着新轴重复32次，
这样我们将 y 沿着新轴重复 32 次，这样得到的张量 Y 的形状为 (32, 10)，
并且 Y[i, :] == y for i in range(0, 32)。现在，我们可以将 X 和 Y 相加，
因为它们的形状相同。

简单的代码示例：

In [34]:
def sugar_add_matrix_and_vector(x_max:np.ndarray,y_min:np.ndarray):
    assert len(x_max.shape)==2
    assert len(y_min.shape)==1
    assert x_max.shape[1]==y_min.shape[0]
    res = x_max.copy()
    for i in range(x_max.shape[0]):
        for j in range(x_max.shape[1]):
            res[i,j]+=y_min[j]
    return res

x = np.array([[1,2],[3,4]])
y = np.array([3,3])
print(sugar_add_matrix_and_vector(x,y))

[[4 5]
 [6 7]]


如果一个张量的形状是 (a, b, ... n, n+1, ... m)，另一个张量的形状是 (n, n+1,... m)，那么通常可以利用广播对它们做两个张量之间的逐元素运算。广播操作会自动应用于从 a 到 n-1 的轴。
下面这个例子利用广播将逐元素的 maximum 运算应用于两个形状不同的张量。

In [44]:
x1 = np.random.random((65,1,2,3))
y1 = np.random.random((2,3))
z = np.maximum(x1,y1)
print(z.shape)

(65, 1, 2, 3)


## 张量点积
点积运算，也叫张量积（tensor product，不要与逐元素的乘积弄混），是最常见也最有用的
张量运算。与逐元素的运算不同，它将输入张量的元素合并在一起。
在 Numpy、Keras、Theano 和 TensorFlow 中，都是用 * 实现逐元素乘积。TensorFlow 中的
点积使用了不同的语法，但在 Numpy 和 Keras 中，都是用标准的 dot 运算符来实现点积。